In [190]:
import pandas as pd
import numpy as np
import scipy
from scipy.stats import norm
from scipy.linalg import solve_banded
from scipy.linalg import solve

### Question 1

In [234]:
# This function uses the vasicek model and calculates the pathwise discount rates using which we need to find bond price
def vasicek_model(T, dt, r0, sigma, k, r_mean):
    N = int(T/dt)
    dW = np.sqrt(dt) * np.random.standard_normal(N)

    # Create an array to store the interest rates
    rt = np.zeros(N+1)
    rt[0] = r0

    # Monte Carlo simulation
    for t in range(1, N+1):
        rt[t] = rt[t-1] + k * (r_mean - rt[t-1]) * dt + sigma * dW[t-1]
    
    return rt

# This function calculates the price of the bond using the pathwise rates, coupons and face value
def bond_price_calc(dt, rt, ct, face, t, T):
    
    # will be used in calculating the discount factor
    rt_cumsum = np.cumsum(rt)
    bond_price = 0
    
    n = int(t/dt)
    N = int(T/dt) - 1
    
    for i in range(N, -1, n-1):
        if ct[i] > 0:
            disc_f = np.exp(-rt_cumsum[i] * dt)
            bond_price = bond_price + disc_f * ct[i]
    
    bond_price += (np.exp(-(rt_cumsum[N] - rt_cumsum[n]) * dt) * face)
    return bond_price

# This function calculates call option price for the given discount rate path. Here nth elements corresponds to the maturity of call option i.e n*dt  = maturity
def call_option_calc(t, T, S, dt, rt, face, strike):
    rt_cumsum = np.cumsum(rt)
    n1 = int(t/dt)
    n2 = int(T/dt)
    N = int(S/dt)
    
    bond_price = (np.exp(-(rt_cumsum[N] - rt_cumsum[n2]) * dt) * face)
    call_price = max((bond_price - strike), 0) * np.exp(-(rt_cumsum[n2] - rt_cumsum[n1]) * dt)
    return call_price

#### (a)  ZC Bond $ \quad r_0 = 5\% \quad \sigma = 10\% \quad \kappa = 0.82 \quad \overline{r} = 0.05 \quad Face = 1000 \quad T = 0.5 years $

In [228]:
r0 = 0.05
r_mean = 0.05
sigma = 0.1
k = 0.82
F = 1000
T = 0.5
dt = 1/252
N = int(T/dt)
bond_prices = np.zeros(10000)
ct = np.zeros(10000)

for i in range(10000):
    rt = vasicek_model(T, dt, r0, sigma, k, r_mean)
    bond_prices[i] = bond_price_calc(dt, rt, ct, F, 0, T)

bond_prices.mean()

975.8895476571282

#### (b) Semi Annual Coupon Bond $ \quad r_0 = 5\% \quad \sigma = 10\% \quad \kappa = 0.82 \quad \overline{r} = 0.05 \quad Face = 1000 \quad T = 4 years \quad C = 30 $

In [330]:
r0 = 0.05
r_mean = 0.05
sigma = 0.1
k = 0.82
F = 1000
T = 4
dt = 1/252
N = int(T/dt)
bond_prices = np.zeros(1000)

semi_annual = int(0.5/dt)
ct = np.zeros(N)
ct[::semi_annual] = 30

for i in range(1000):
    rt = vasicek_model(T, dt, r0, sigma, k, r_mean)
    bond_prices[i] = bond_price_calc(dt, rt, ct, F, 0, T)
    
bond_prices.mean()

1039.5504319073561

#### (c) Call option on ZC bond with $ K = 980 \quad Expiry = 0.25 $

In [331]:
r0 = 0.05
r_mean = 0.05
sigma = 0.1
k = 0.82
F = 1000
T = 0.5
dt = 1/252
N = int(T/dt)
ct = np.zeros(10000)

K = 980
call_mat = 0.25
n = int(call_mat/dt)
T_t = T-call_mat

B = 1/k * (1 - np.exp(-k * T_t))
A = np.exp((r_mean - (sigma**2 / (2 * k**2))) * (B - T_t) - ((sigma**2 / (4 * k)) * B**2))

call_options = np.zeros(10000)

for i in range(10000):
    rt = vasicek_model(T, dt, r0, sigma, k, r_mean)
    bond_price = F * A * np.exp(-B * rt[n])
    call_option1 = (bond_price - K) * np.exp(-np.cumsum(rt)[n] * dt)
    call_option2 = call_option_calc(0, call_mat, T, dt, rt, F, K)
    call_options[i] = (call_option1 + call_option2) * 0.5 # control variate

call_options.mean()

8.595094530228053

#### (d) Call option for Coupon Bond with $ K = 980 \quad Expiry = 0.25 $

In [406]:
r0 = 0.05
r_mean = 0.05
sigma = 0.1
k = 0.82
F = 1000
T = 4
dt = 1/252
N = int(T/dt)

semi_annual = int(0.5/dt)
ct = np.zeros(N)
ct[::semi_annual] = 30
ct[0] = 0

option_prices = np.zeros(10000)

for i in range(10000):
    rt = vasicek_model(T, dt, r0, sigma, k, r_mean)
    rt_cumsum = np.cumsum(rt)
    positive_ct_indices = np.where(ct > 0)[0]
    FV = F + np.sum(ct[positive_ct_indices] * np.exp((rt_cumsum[-1] - rt_cumsum[positive_ct_indices]) * dt))
    option_prices[i] = call_option_calc(0, 0.25, T, dt, rt, FV, 980)
option_prices.mean()

94.97931619051538

#### (e) Call option for Coupon Bond with $ K = 980 \quad Expiry = 0.25 $ using explicit formula

In [410]:
def bond_price(rates, dt, coupons, F):
    discount_factors = np.exp(-rates*dt)
    return np.sum(coupons*discount_factors) + F*discount_factors[-1]

def option_price(expiry, T, dt, K, r0, r_mean, k, sigma, F, cash_flows):
    num_simulations = 1000
    total_option_prices = []
    t = dt * np.where(cash_flows > 0)[0]
    
    for i in range(num_simulations):
        rt = vasicek_model(T, dt, r0, r_mean, k, sigma)
        rt_cumsum = np.cumsum(rt)
        df = np.zeros(len(t))
        for j in range(len(t)):
            df[j] = np.exp(-rt_cumsum[int(t[j]/dt)] * dt)
        df_expiry = np.exp(-rt_cumsum[int(expiry/dt)] * dt)
        cf = cash_flows[np.where(cash_flows > 0)]
        sigma_p = sigma * (1 - np.exp(-k*(t - expiry))) / k * np.sqrt((1 - np.exp(-2*k*expiry)) / (2*k))
        d1 = (np.log(cf * df / (K * df_expiry)) + sigma_p**2 / 2) / sigma_p
        d2 = d1 - sigma_p
        option_prices = cf * df * norm.cdf(d1) - K * df_expiry * norm.cdf(d2)
        total_option_prices.append(np.sum(cf * option_prices))
    return np.mean(total_option_prices)

T = 4
dt = 1/252
N = int(T/dt)
r0 = 0.05
r_mean = 0.05
k = 0.82
sigma = 0.1
F = 1000
K = 980

semi_annual = int(0.5/dt)
coupons = np.zeros(N + 1)
coupons[::semi_annual] = 30
coupons[0] = 0
coupons[1008] += F

call_option_price = option_price(0.25, T, dt, K, r0, r_mean, k, sigma, F, coupons)
call_option_price

97.35273129679005

### Question 2

In [245]:
# This function uses the CIR model and calculates the pathwise discount rates using which we need to find bond price
def cir_model(T, dt, r0, sigma, k, r_mean):
    N = int(T/dt)
    dW = np.sqrt(dt) * np.random.standard_normal(N)

    # Create an array to store the interest rates
    rt = np.zeros(N+1)
    rt[0] = r0

    # Monte Carlo simulation
    for t in range(1, N+1):
        rt[t] = rt[t-1] + k * (r_mean - rt[t-1]) * dt + sigma * np.sqrt(rt[t-1]) * dW[t-1]
        if rt[t] < 0:
            rt[t] = 0
    
    return rt

#### (a)  ZC Bond Call Option $ \quad r_0 = 5\% \quad \sigma = 12\% \quad \kappa = 0.92 \quad \overline{r} = 0.055 \quad Face = 1000 \quad Bond Maturity = 1 years \quad K = 980 \quad Expiry (T) = 0.5 $

In [402]:
r0 = 0.05
r_mean = 0.055
sigma = 0.12
k = 0.92
F = 1000
S = 1.0 # bond maturity
dt = 1/252
ct = np.zeros(1000)

K = 980
T = 0.5 # call expiry
M = int(T/dt)

call_options = np.zeros(1000)
for i in range(1000):
    rt = cir_model(S, dt, r0, sigma, k, r_mean)
    call_options[i] = call_option_calc(0, T, S, dt, rt, F, K)
call_options.mean()

0.3780133430746902

####  (b) ZC Bond Call Option using IFD $ \quad r_0 = 5\% \quad \sigma = 12\% \quad \kappa = 0.92 \quad \overline{r} = 0.055 \quad Face = 1000 \quad S = 1 years \quad K = 980 \quad Expiry (T) = 0.5 $

In [403]:
# Parameters
N = 1000  
M = 126
S = 1.0  # bond maturity
T = 0.5
r0 = 0.05 
sigma = 0.12  # volatility
r_mean = 0.055 # long-term mean rate
r_max =  r_mean + (10 * sigma * r_mean)  # maximum rate
r_min = 0.0  # minimum rate
k = 0.92  # mean reversion speed
F = 1000
K = 980  # strike price

# Grid setup
dt = T / M  # time step
dr = (r_max - r_min) / N  # rate step
t_grid = np.linspace(0, T, M + 1)  # time grid
r_grid = np.linspace(r_min, r_max, N + 1)  # rate grid

# Initialize option price matrix
C = np.zeros((M + 1, N + 1))

# Terminal condition
C[-1, :] = np.maximum(F * np.exp(-r_grid * (S - T)) - K, 0)

# Matrix coefficients
j = np.arange(0, N+1)
Pu = -0.5 * dt * (j*sigma**2/dr + k*r_mean/dr - (k*j)/2)
Pm = dt * (1/dt + j*sigma**2/dr + r_grid)
Pd = -0.5 * dt * (j*sigma**2/dr - k*r_mean/dr + (k*j)/2)

for i in reversed(range(M)):
    # Vector B
    B = np.zeros(N+1)
    B[0] = Pu[N] * C[i, N]  # boundary condition
    B[-1] = Pd[1] * C[i, 0]  # boundary condition

    A_upper = np.zeros(N+1)
    A_middle = np.zeros(N+1)
    A_lower = np.zeros(N+1)
    
    A_upper[1:] = Pu[1:]
    A_middle = Pm
    A_lower[:-1] = Pd[:-1]
    
    A_banded = np.vstack((A_upper, A_middle, A_lower))

    # Solve the system
    C[i, :] = solve_banded((1, 1), A_banded, C[i+1, :] + B)
    C[i, 0] = F - K  # boundary condition
    C[i, -1] = 0  # boundary condition

# Price of the call option at time 0
C[0, int(r0/dr)] 

0.4120747264274198

#### (c) Calculating Call option price using explicit formula for CIR bonds

In [397]:
# Parameters
N = 1000  
M = 126
S = 1.0  # bond maturity
T = 0.5
dt = T / M  # time step
r0 = 0.05 
sigma = 0.12  # volatility
r_mean = 0.055 # long-term mean rate
k = 0.92  # mean reversion speed
F = 1000
K = 980  # strike price

h1 = np.sqrt(k**2 + 2*sigma**2)
h2 = (k+h1)/2
h3 = (2*k*r_mean)/sigma**2
T_t = S-T

A_num = h1 * np.exp(h2 * T_t)
A_denom = h2 * (np.exp(h1 * T_t)-1) + h1
A = (A_num/A_denom) ** h3

B_num = (np.exp(h1 * T_t)-1)
B_denom = A_denom
B = B_num/B_denom

option_prices = np.zeros(1000)
for i in range(1000):
    rt = cir_model(S, dt, r0, sigma, k, r_mean)
    bond_price = F * A * np.exp(-B * rt[M])
    option_prices[i] = max((bond_price - K), 0) * np.exp(-np.cumsum(rt)[M] * dt)
    
option_prices.mean()

0.44109590009085803

### Question 3

In [241]:
# This function uses the G2++ model and calculates the pathwise discount rates using which we need to find bond price
def g2_model(T, dt, a, sigma, b, eta, phi, r0):
    N = int(T/dt)
    dW1 = np.sqrt(dt) * np.random.standard_normal(N)
    dW2 = rho * dW1 + np.sqrt(1 - rho**2) * np.random.standard_normal(N) * np.sqrt(dt)
    
    x = np.zeros(N + 1)
    y = np.zeros(N + 1)
    r = np.zeros(N + 1)
    r[0] = r0
    
    for i in range(1, N+1):
        dx = -a * x[i-1] * dt + sigma * dW1[i-1]
        dy = -b * y[i-1] * dt + eta * dW2[i-1]
        x[i] = x[i-1] + dx
        y[i] = y[i-1] + dy
        r[i] = x[i]+y[i]+phi
    
    return r

In [395]:
# Parameters
T = 0.5  # Put option expiration
S = 1.0  # Bond maturity
r0 = 0.03  # Initial interest rate
phi = 0.03  # Constant term
a = 0.1
b = 0.3
sigma = 0.03
eta = 0.08
rho = 0.7
K = 950  # Strike price
F = 1000  # Face value
num_simulations = 10000

option_prices = np.zeros(num_simulations)
bond_prices_T = np.zeros(num_simulations)
bond_prices_S = np.zeros(num_simulations)
ct = np.zeros(int(S/dt))

for i in range(num_simulations):
    rt = g2_model(S, dt, a, sigma, b, eta, phi, r0)
    rt_cumsum = np.cumsum(rt)
    D_T = (np.exp(-rt_cumsum[int(T/dt)] * dt))
    D_ST = (np.exp(-(rt_cumsum[int(S/dt)] - rt_cumsum[int(T/dt)]) * dt))
    D_S = (np.exp(-rt_cumsum[int(S/dt)] * dt))

    # Compute the European put option price
    Put_MC1 = max(K - F * D_ST, 0) * D_T
    Put_MC2 = 0.25 * max(K - F * D_S/D_T, 0) * D_T # antithetic variate
    option_prices[i] = (Put_MC1 + Put_MC2)/2
    bond_prices_T[i] = D_T
    bond_prices_S[i] = D_S
    
Put_MC = option_prices.mean()
P_T = bond_prices_T.mean()
P_S = bond_prices_S.mean()

In [396]:
# Calculate sigma for the explicit formula
sigma_exp = np.sqrt((sigma**2 / (2 * a**3) * (1 - np.exp(-a * (S - T)))**2 * (1 - np.exp(-2 * a * (T))) 
                     + eta**2 / (2 * b**3) * (1 - np.exp(-b * (S - T)))**2 * (1 - np.exp(-2 * b * (T))) 
                     + 2 * rho * sigma * eta / (a * b * (a + b)) * (1 - np.exp(-a * (S - T))) * (1 - np.exp(-b * (S - T))) * (1 - np.exp(-(a + b) * (T)))))

# Computea the European put option price using the explicit formula
d1 = (np.log(K * P_T / (F * P_S)) + sigma_exp**2 / 2) / sigma_exp
d2 = d1 - sigma_exp
Put_exp = -F * P_S * norm.cdf(d2) + K * P_T * norm.cdf(d1)

print(f"European Put Option price (Monte Carlo): {Put_MC}")
print(f"European Put Option price (Explicit Formula): {Put_exp}")

European Put Option price (Monte Carlo): 1.9253005671751882
European Put Option price (Explicit Formula): 1.9054888399475942
